# Install Dependencies

In [1]:
# Install uv package manager
!pip install uv

# Verify install
!uv --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 50.5 MB/s eta 0:00:00
uv 0.6.16


In [2]:
# Clone the repo (jackdaus version has uv support)
!git clone https://github.com/jackdaus/egolifter.git
%cd egolifter
# Checkout the colab branch. I'm developing compatibility with colab.
!git checkout colab

Cloning into 'egolifter'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 230 (delta 74), reused 183 (delta 52), pack-reused 17 (from 1)
Receiving objects: 100% (230/230), 4.72 MiB | 5.63 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/egolifter
Branch 'colab' set up to track remote branch 'colab' from 'origin'.
Switched to a new branch 'colab'


In [3]:
# Install packages. This might take a couple of minutes.
!uv sync

Streaming output truncated to the last 5000 lines.
nvidia-cusparse-cu12 ------------------------------ 121.00 MiB/197.84 MiB
triton     ------------------------------ 123.26 MiB/199.76 MiB
nvidia-cufft-cu12 ------------------------------ 119.98 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 120.21 MiB/346.60 MiB
open3d     ------------------------------ 123.61 MiB/426.94 MiB
nvidia-cudnn-cu12 ------------------------------ 120.26 MiB/633.96 MiB
⠼ Preparing packages... (229/239)
plotly     ------------------------------ 9.87 MiB/14.12 MiB
nvidia-cusolver-cu12 ------------------------------ 120.97 MiB/122.01 MiB
nvidia-nccl-cu12 ------------------------------ 120.10 MiB/179.91 MiB
nvidia-cusparse-cu12 ------------------------------ 121.00 MiB/197.84 MiB
triton     ------------------------------ 123.64 MiB/199.76 MiB
nvidia-cufft-cu12 ------------------------------ 120.26 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 120.52 MiB/346.60 MiB
open3d     --

In [8]:
!pwd

/content/egolifter


In [18]:
# Note that in order to use the uv virtual environment, we must activate it within
# the cell. (Colab was giving me trouble creating a uv based kernel...)

# To illustrate, notice that the output of the two commands shows different locations
# of the python exectuable in use.
!source .venv/bin/activate; which python

!which python

/content/egolifter/.venv/bin/python
/usr/local/bin/python


In [19]:
# Unfortunately, we must do something convoluted. We use Jupyter notebook magic
# to create a bash script. We then activate the venv. We then run some python code.
%%bash
source .venv/bin/activate

python - <<'PY'
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
PY

PyTorch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA is available: True


In [113]:
# Set environment variables (adpated from setup_env.bash in original egolifter)
import os
os.environ.update({
    "EGOLIFTER_PATH":                 "/content/egolifter",
    "GSA_PATH":                       "/content/egolifter/Grounded-Segment-Anything",
    "SAM_CHECKPOINT_PATH":            "/content/egolifter/Grounded-Segment-Anything/sam_vit_h_4b8939.pth",
    "GROUNDING_DINO_CHECKPOINT_PATH": "/content/egolifter/Grounded-Segment-Anything/groundingdino_swint_ogc.pth",
    "SAM_ENCODER_VERSION":            "vit_h",
    "GROUNDING_DINO_CONFIG_PATH":     "/content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
    "EFFICIENTSAM_PATH":              "/content/egolifter/Grounded-Segment-Anything/EfficientSAM",
    "TAG2TEXT_PATH":                  "/content/egolifter/Grounded-Segment-Anything/Tag2Text",
    "ADT_DATA_ROOT":                  "adt",
    "ADT_PROCESSED_ROOT":             "adt_processed",
    "AM_I_DOCKER":                    "False",
    "BUILD_WITH_CUDA":                "True",
    "TAG2TEXT_CHECKPOINT_PATH":       "/content/egolifter/Grounded-Segment-Anything/Tag2Text/tag2text_swin_14m.pth",
    "RAM_CHECKPOINT_PATH":            "/content/egolifter/Grounded-Segment-Anything/Tag2Text/ram_swin_large_14m.pth",
})

In [39]:
# Set up Grounded-Segment-Anything
%%bash
!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything.git

fatal: destination path 'Grounded-Segment-Anything' already exists and is not an empty directory.
Using Python 3.11.12 environment at: /usr
Resolved 1 package in 622ms
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/segment_anything
      Built segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/segment_anything
Prepared 1 package in 638ms
Installed 1 package in 1ms
 + segment-anything==1.0 (from file:///content/egolifter/Grounded-Segment-Anything/segment_anything)
Using Python 3.11.12 environment at: /usr
Resolved 57 packages in 30.97s
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/GroundingDINO
  × Failed to build `groundingdino @
  │ file:///content/egolifter/Grounded-Segment-Anything/GroundingDINO`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_editable` failed (exit status: 1)

      [stdout]
      Compiling with CUDA
      running editable_wheel
      creat

In [41]:
# %%bash
# export AM_I_DOCKER=False
# export BUILD_WITH_CUDA=True
# uv pip install -e Grounded-Segment-Anything/segment_anything

Using Python 3.11.12 environment at: /usr
Resolved 1 package in 576ms
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/segment_anything
      Built segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/segment_anything
Prepared 1 package in 557ms
Uninstalled 1 package in 0.37ms
Installed 1 package in 0.85ms
 ~ segment-anything==1.0 (from file:///content/egolifter/Grounded-Segment-Anything/segment_anything)


In [61]:
!uv add Grounded-Segment-Anything/segment_anything

Resolved 247 packages in 1.31s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
   Building segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
      Built segment-anything @ file:///content/egolifter/Grounded-Segment-Anything/s
Prepared 1 package in 586ms
Installed 1 package in 1ms
 + segment-anything==1.0 (from file:///content/egolifter/Grounded-Segment-Anything/segment_anything)


In [47]:
!uv pip install numpy ninja

Using Python 3.11.12 environment at: /usr
Resolved 2 packages in 78ms
Installed 1 package in 3ms
 + ninja==1.11.1.4


In [63]:
%%bash
source .venv/bin/activate
python - <<'PY'
import torch
# import groundingdino
print("torch", torch.__version__, "cuda", torch.version.cuda)
# print("GroundingDINO import OK:", groundingdino.__version__)
PY


torch 2.5.1+cu124 cuda 12.4


In [75]:
%%bash
source .venv/bin/activate
uv pip uninstall torch torchvision torchaudio

Using Python 3.11.12 environment at: /usr


In [78]:
# Make quick-and-dirty fix for issue with GroundingDino not working in Colab (early 2025).
# See: https://github.com/IDEA-Research/Grounded-Segment-Anything/issues/550
%cd /content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn
!sed -i 's/value.type()/value.scalar_type()/g' ms_deform_attn_cuda.cu
!sed -i 's/value.scalar_type().is_cuda()/value.is_cuda()/g' ms_deform_attn_cuda.cu

/content/egolifter/Grounded-Segment-Anything/GroundingDINO/groundingdino/models/GroundingDINO/csrc/MsDeformAttn


In [104]:
%cd ~/../content
%cd egolifter

/content
/content/egolifter


In [105]:
!uv add Grounded-Segment-Anything/GroundingDINO
# uv pip install --upgrade "diffusers[torch]"

Resolved 253 packages in 1.49s
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:///content/egolifter/Grounded-Segment-Anything/Grou
   Building groundingdino @ file:/

In [22]:
# Download model weights
%cd Grounded-Segment-Anything/
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
%cd ..

/content/egolifter/Grounded-Segment-Anything
--2025-04-23 04:57:14--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.111, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   122MB/s    in 16s     

2025-04-23 04:57:31 (154 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]

--2025-04-23 04:57:31--  https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-p

# Download Data

First, upload your `ADT_download_urls.json` file. Then, run the code below.

In [23]:
# IMPORTANT: Make sure you have uploaded your file `ADT_download_urls.json`!
#            See original repo for instructions on how to get that.

%%bash
source .venv/bin/activate

python - <<'PY'

import os
import shutil

# Define data directories
ADT_DATA_ROOT = "adt"
ADT_PROCESSED_ROOT = "adt_processed"

# Create directories if they don't exist
os.makedirs(ADT_DATA_ROOT, exist_ok=True)
os.makedirs(ADT_PROCESSED_ROOT, exist_ok=True)

# Copy the download URLs JSON file
source_path = "/content/egolifter/ADT_download_urls.json"  # Update if needed
destination_path = os.path.join(ADT_DATA_ROOT, "ADT_download_urls.json")
shutil.copy(source_path, destination_path)

PY

In [107]:
# Sanity check to see if environment variables make their way into the uv venv
%%bash
source .venv/bin/activate

python - <<'PY'
import os
print({os.environ['ADT_DATA_ROOT']})
PY

{'adt'}


In [27]:
# Copy the vignette images to the dataset
!cp assets/vignette_imx577.png ${ADT_DATA_ROOT} # Vignette image for the RGB camera
!cp assets/vignette_ov7251.png ${ADT_DATA_ROOT} # Vignette image for the SLAM camera

In [35]:
# Move the vignette files
import shutil

# Define the destination paths within the data directory
vignette_rgb_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_imx577.png")
vignette_slam_destination = os.path.join(os.environ['ADT_DATA_ROOT'], "vignette_ov7251.png")

# Copy the vignette images
shutil.copy("assets/vignette_imx577.png", vignette_rgb_destination)
shutil.copy("assets/vignette_ov7251.png", vignette_slam_destination)

'adt/vignette_ov7251.png'

In [36]:
# Define the scene names in a Python list. For now, we just have one sample scene.
scene_names_new = ["Apartment_release_golden_skeleton_seq100_10s_sample_M1292"]
scene_names     = ["Apartment_release_golden_skeleton_seq100_10s_sample"]

# Loop through the scene names and execute the download command for each
for scene_name in scene_names_new:
    !uvx --from projectaria-tools aria_dataset_downloader \
        -c adt/ADT_download_urls.json \
        -o adt/ \
        -d 0 1 2 3 6 7 \
        -l {scene_name}

⠙ Preparing packages... (0/3)
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------     0 B/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------     0 B/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 14.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 14.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 30.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 30.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 46.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 46.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ------------------------------ 62.90 KiB/140.49 KiB
⠙ Preparing packages... (0/3)
charset-normalizer ----------------

# Process Data (Part 1)

In [37]:
# Process the images to be in a format appropriate for 3dgs
for scene_name in scene_names:
  !uv run python scripts/process_adt_3dgs.py \
    --data_root adt \
    --output_root adt_processed \
    --sequence_name {scene_name}

Processing sequence adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292
[AriaDigitalTwinDataPathsProvider][WARNING]: no depth image data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/depth_images_with_skeleton.vrs
[AriaDigitalTwinDataPathsProvider][WARNING]: no synthetic video data found at adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/synthetic_video.vrs
[MpsDataPathsProvider][WARNING]: Hand tracking folder (adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/mps/hand_tracking) does not exist in MPS root folder, not loading wrist and palm poses.
[ProgressLogger][INFO]: 2025-04-23 05:05:01: Opening adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs...
[MultiRecordFileReader][DEBUG]: Opened file 'adt/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb act

# Train on Vanilla 3DGS Pipeline

This is a first test of training on the vanilla 3DGS pipeline.

In [ ]:
# Grab the first (and only?) scene that was downloaded/processed, above.
# It will ask if you want to log into wandb to visualize training progress/logs
scene_name = scene_names[0]

# Run the code. In Colab, we must limit number of worker threads to 2.
!uv run python train_lightning.py \
  scene.scene_name={scene_name} \
  scene.data_root={ADT_PROCESSED_ROOT} \
  exp_name=3dgs \
  output_root=./output/adt \
  wandb.project=egolifter_adt \
  scene.num_workers=2

/content/egolifter/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'train': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
Working directory : /content/egolifter
Output directory  : /content/egolifter/output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/vanilla_3dgs/2025-04-22_04-40-57
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jackdaus (jackdaus-george-mason-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in ./output/adt/Apartment_release_golden_skeleton_seq100_10s_sample/vanilla_3dgs/wandb/run-20250422_044058-doyn7tl2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run 3dgs
wandb: ⭐️ View proje

# Process Data (Part 2)

## Segmentation

In [ ]:
# Generate the SAM segmentation results

# TEMPORARY: Hard-coded a single scene name. But we might want to loop through these...
!uv run python scripts/generate_gsa_results.py \
  -i adt_processed/Apartment_release_golden_skeleton_seq100_10s_sample \
  --class_set none \
  --sam_variant sam \
  --max_longer_side 512 \
  --no_clip

Skipping tagging and detection models. 
Found global_points.csv.gz file, assuming Aria data set!
 29% 78/267 [09:24<22:32,  7.16s/it]

## Generate evaluation target for query-based segmentation

### generate_2dseg_query

In [ ]:
# TODO

### generate_2dseg_query_sample

In [ ]:
# TODO

### generate_3dbox_query

In [ ]:
# TODO

# EgoLifter (full method)

In [ ]:
# TODO
# Take the following scene as an example. Change SCENE_NAME if needed.
SCENE_NAME=Apartment_release_golden_skeleton_seq100_10s_sample

# EgoLifter (full method)
!uv run python train_lightning.py \
    scene.scene_name=${SCENE_NAME} \
    scene.data_root=${ADT_PROCESSED_ROOT} \
    model=unc_2d_unet \
    model.unet_acti=sigmoid \
    model.dim_extra=16 \
    lift.use_contr=True \
    exp_name=egolifter \
    output_root=./output/adt \
    wandb.project=egolifter_adt